In [8]:
import pandas as pd
import json
import requests
url = 'https://raw.githubusercontent.com/brearenee/NLP-Project/main/dataset/StarTrekDialogue_v2.json'
response = requests.get(url)

##This CodeBlock is thanks to ChatGPT :-) 
if response.status_code == 200:
    json_data = json.loads(response.text)
    lines = []
    characters = []
    episodes = []
  
    # extract the information from the JSON file for the "TNG" series
    for series_name, series_data in json_data.items():
        if series_name == "TNG": 
            for episode_name, episode_data in series_data.items():
                for character_name, character_lines in episode_data.items():
                    for line_text in character_lines:
                        lines.append(line_text)
                        characters.append(character_name)
                        episodes.append(episode_name)
                     
    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'Line': lines,
        'Character': characters,
    })

    # Remove duplicate lines, keeping the first occurrence (preserving the original order)
    df = df.drop_duplicates(subset='Line', keep='first')

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    
##Remove Outliers (Characters with less than 1000 lines)
character_counts = df['Character'].value_counts()
characters_to_remove = character_counts[character_counts < 1000].index
df = df[~df['Character'].isin(characters_to_remove)]

##Print Value Count. 
print(df['Character'].value_counts())


Character
PICARD     10798
RIKER       6454
DATA        5699
LAFORGE     4111
WORF        3185
CRUSHER     2944
TROI        2856
WESLEY      1206
Name: count, dtype: int64


# BERT

In [9]:
#https://www.analyticsvidhya.com/blog/2021/12/multiclass-classification-using-transformers/


#Split the data 
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

#Converting our Character column into Categorical data
encoded_dict = {'PICARD':0,'RIKER':1, 'DATA':2, 'LAFORGE':3, 
                'WORF':4, 'CRUSHER':5, 'TROI':6,'WESLEY':7}
train_df['Character'] = train_df.Character.map(encoded_dict)
val_df['Character'] = val_df.Character.map(encoded_dict)

train_df.head()




,Line,Character
41033,The upper portion of the apparatus seems\r to ...,2
19647,how long before they cross over into Federatio...,1
20640,"You know, this might work. We can't change the...",3
16952,And you conclude because of this that I am imp...,2
11530,I was thinking the same thing about you. In al...,1


In [10]:
val_df.head()

,Line,Character
48560,"If you had to give this feeling a name, what w...",6
47262,I'm sure you all understand that in light of w...,5
7425,"Somehow, and there is limited information on t...",2
29414,But our sensors were malfunctioning. Our probe...,1
33815,The signal ended abruptly at oh four five five...,1


In [11]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_df.Character)
y_test = to_categorical(val_df.Character)

#We have successfully processed our Sentiment column( target); 
#now, it’s time to process our input text data using a tokenizer.

In [12]:
import transformers

#Loading Model and Tokenizer from the transformers package 

from transformers import AutoTokenizer,TFBertModel
#bert-base-uncased is another possible one
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
#TFBertModel = pretrained BERT model for Tensor Flow
bert = TFBertModel.from_pretrained('bert-base-cased')

#Input Data Modeling

#Before training, we need to convert the input textual data into 
#BERT’s input data format using a tokenizer.
#Since we have loaded bert-base-cased, 
#so tokenizer will also be Bert-base-cased.
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=train_df.Line.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=val_df.Line.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


#Hereafter data modelling, the tokenizer will return a dictionary (x_train) containing ‘Input_ids’, ‘attention_mask’ as key for their respective
#data.

input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Model Building

In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(6,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

KeyboardInterrupt: 

# Model Compilation

Defining learning parameters and compiling the model.

In [ ]:

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Model Training

In [ ]:
#We have the model ready with x_train, y_train. You can now train the model.
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)


# Model Evaluation

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.Character

from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))